![](images/mk.png)

<h1><center>ONLINE SUPPLEMENT</center></h1>

John McLevey & Reid-McIlroy-Young. **Introducing *metaknowledge*: Software for Computational Research in Information Science, Science of Science, and Network Analysis.** *Journal of Informetrics*. XX(XX):XX-XX.

<h1><center>Part 3: Text Analysis</center></h1>

This supplementary notebook was prepared by [Steve McColl](http://networkslab.org/) (NetLab, University of Waterloo), Dr. [John McLevey](http://www.johnmclevey.com/) (University of Waterloo), and [Reid McIlroy-Young](http://reidmcy.com/) (University of Chicago). The code in this notebook is current as of *metaknowledge* version 3.1.1.

In [13]:
import metaknowledge as mk
from stop_words import get_stop_words
from nltk.tokenize import RegexpTokenizer
import seaborn as sns
import numpy
import matplotlib as plt
import pandas
import os

# Imports for gensim.
import gensim
from gensim import corpora, models

# Imports for pyLDAvis.
import pyLDAvis.gensim as gensimvis
import pyLDAvis

# Imports for sklearn.
# from __future__ import print_function
# import sklearn
# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# from sklearn.decomposition import NMF, LatentDirichletAllocation

sns.set_style(style="white") # change the default background plot colour
sns.set(font_scale=.7)

plt.rc("savefig", dpi=300) # improve default resolution of graphics

os.chdir('.')

/Users/johnmclevey/anaconda/lib/python3.4/site-packages/matplotlib/__init__.py:878: UserWarning: axes.color_cycle is deprecated and replaced with axes.prop_cycle; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [14]:
RC = mk.RecordCollection('raw_data/imetrics/', cached = True)

# Topic Models


* Create topic models using the Gensim package or Scikit-Learn.
* Create interactive visualizations using Gensim models and pyLDAvis.

In [16]:
# Transform the record collection into a format for use with natural language processing applications.
raw = RC.forNLP('generated_datasets/topic_model/topic_model.csv', lower=True, removeNumbers=True,
         removeNonWords=True, removeWhitespace=True, removeCopyright=False)

In [17]:
# Conver the raw text into a list.
documents = raw['abstract']

## SKLearn

* Basic topic models using code from 
[this tutorial](http://scikit-learn.org/stable/auto_examples/applications/topics_extraction_with_nmf_lda.html) written by Olivier Grisel, Lars Buitinck, and Chyi-Kwei Yau.

In [18]:
# For use with SKlearn, convert the raw text to a numpy array.
docs = numpy.asarray(documents)

In [19]:
# Increasing the number of features will give a better model, but it may increase the runtime.
features = 1000
topics = 50
top_words = 20

In [20]:
# Initialize the tokenizer.
tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2,
                                   max_features=features,
                                   stop_words='english')

In [21]:
# Tokenize the documents.
tfidf = tfidf_vectorizer.fit_transform(docs)

In [22]:
# Define the output function, code exactly as shown in tutorial.
def print_top_words(model, feature_names, top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-top_words - 1:-1]]))
    print()

In [23]:
# Extract the features (tokens) for the models.
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=features,
                                stop_words='english')

In [24]:
tf = tf_vectorizer.fit_transform(docs)

In [25]:
# Fit the Non-negative matrix factorization model.
nmf = NMF(n_components=topics, random_state=1, alpha=.1, l1_ratio=.5).fit(tfidf)

TypeError: __init__() got an unexpected keyword argument 'l1_ratio'

In [60]:
# Print the list of topics and their contents. 
# Note that the constraints can be modified by changing the number of topics or number of words in each topic.
tfidf_feature_names = tfidf_vectorizer.get_feature_names()
print_top_words(nmf, tfidf_feature_names, top_words)

Topic #0:
papers citation citations cited number authors published articles highly paper references author citing years counts received year publication average total
Topic #1:
information use behavior systems users retrieval seeking article study design social model library internet user science digital online health theory
Topic #2:
journals journal impact factor articles citation published factors jcr subject jif isi oa reports categories year citing sciences citations editorial
Topic #3:
patent patents technology technological innovation rd technologies firms patenting companies citations industry knowledge development applications analysis indicators nanotechnology paper value
Topic #4:
search web users query queries user engines results searching retrieval engine pages documents tasks terms sites interface task relevance searchers
Topic #5:
collaboration international collaborative coauthorship network networks countries collaborations chinese domestic impact coauthored instituti

In [66]:
# Extract topics for the LDA model, and fit the model.
lda = LatentDirichletAllocation(n_topics=topics, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50.0,
             max_doc_update_iter=100, max_iter=5, mean_change_tol=0.001,
             n_jobs=1, n_topics=10, perp_tol=0.1, random_state=0,
             topic_word_prior=None, total_samples=1000000.0, verbose=0)

In [67]:
# Print the list of topics and their contents. 
# Note that the constraints can be modified by changing the number of topics or number of words in each topic.
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, top_words)

Topic #0:
data web patent analysis technology patents research technological study new results using model paper innovation used based information knowledge approach
Topic #1:
research countries collaboration scientific international science output national institutions universities china chinese production country paper publications funding performance university study
Topic #2:
authors number publications productivity researchers author publication scientists study results research total academic medical authorship university period differences scientific data
Topic #3:
information knowledge study use research social article behavior online systems work theory digital model design factors library internet findings understanding
Topic #4:
journals citation articles papers citations cited published journal impact number science publication research references years publications scientific article year citing
Topic #5:
citation index number hindex different model scientific paper distri


## Gensim

* For model coefficients and more analysis options.
* Code adapted from [the gensim tutorial](https://radimrehurek.com/gensim/tutorial.html) by Radim Řehůřek.
* Visualizations created using pyLDAvis, using code from the [notebooks]('https://github.com/bmabey/pyLDAvis/tree/master/notebooks') by Ben Mabey.

In [26]:
# Create a list of stopwords, using the stopwords package.
stopwords = get_stop_words('en')

In [27]:
# Initialize the tokenizer.
tokenizer = RegexpTokenizer(r'\w+')

In [28]:
# Initialize a list, we will save tokens here.
tokens = []

In [29]:
# Iterate over the documents list, and tokenize and save each entry.
for l in documents:
    token = tokenizer.tokenize(l)
    tokens.append(token)

In [30]:
# Initialize a list, we will save cleaned tokens here.
cleaned_tokens = []

In [31]:
# Keep tokens only if they do not appear in the list of stopwords.
for l in tokens:
    cleaned_tokens.append([i for i in l if not i in stopwords])

In [32]:
# Create dictionary from the cleaned tokens.
dictionary = corpora.Dictionary(cleaned_tokens)

In [33]:
# Convert the cleaned tokens into a numpy array.
array = numpy.asarray(cleaned_tokens)

In [34]:
# Train the corpus using the array, creating a bag-of-words that contains each word in the array.
corpus = [dictionary.doc2bow(word) for word in array]

In [35]:
# Generate the LDA model using gensim.
ldamodel = gensim.models.ldamodel.LdaModel(corpus=corpus, num_topics=50, id2word = dictionary, passes=20)

In [36]:
# The format for printing 50 topics is not very good.
ldamodel.print_topics()

[(47,
  '0.071*"search" + 0.025*"users" + 0.024*"results" + 0.023*"user" + 0.020*"retrieval" + 0.018*"query" + 0.014*"queries" + 0.011*"performance" + 0.010*"system" + 0.010*"ir"'),
 (11,
  '0.041*"growth" + 0.037*"time" + 0.023*"period" + 0.019*"scientific" + 0.013*"publications" + 0.012*"evolution" + 0.010*"years" + 0.010*"literature" + 0.010*"increase" + 0.010*"development"'),
 (1,
  '0.023*"winners" + 0.023*"creative" + 0.019*"nobel" + 0.016*"xml" + 0.015*"coauthorships" + 0.015*"heterogeneous" + 0.014*"triple" + 0.013*"helix" + 0.012*"graphs" + 0.012*"prize"'),
 (21,
  '0.050*"analysis" + 0.021*"cocitation" + 0.020*"research" + 0.019*"using" + 0.018*"clusters" + 0.013*"study" + 0.013*"science" + 0.013*"two" + 0.012*"areas" + 0.012*"structure"'),
 (23,
  '0.038*"ratios" + 0.033*"manufacturers" + 0.028*"correspondence" + 0.021*"statements" + 0.021*"bilateral" + 0.018*"benchmark" + 0.017*"multiauthored" + 0.016*"astronomy" + 0.013*"collaborators" + 0.013*"earth"'),
 (12,
  '0.126*"un

In [39]:
# You also have the option of saving the corpus, dictionary, and model.
# MmCorpus.serialize('paper_abstracts.mm', corpus)
dictionary.save('paper_abstracts.dict')
ldamodel.save('paper_abstracts_lda.model')

### Using the model created with Gensim for an pyLDAvis interactive visualization

In [40]:
# Prepare the visualization data.
vis_data = gensimvis.prepare(ldamodel, corpus, dictionary)

In [41]:
# Visualize the topic model.
pyLDAvis.display(vis_data)